In [12]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train (1).csv


In [13]:
#import pandas to deal with dataset
import pandas as pd
import numpy as np

In [14]:
#Read the data
data = pd.read_csv('train.csv')

In [16]:
#take a look on data
data

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [17]:
#look if there is a NAN value
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


# Preprocessing (Cleaning)

In [18]:
#import string library to deal with text data
import string

In [19]:
#import regular library for replace package
import re

In [20]:
from nltk.corpus import stopwords

In [21]:
from nltk.tokenize import wordpunct_tokenize

In [22]:
def text_clean(txt) :

  txt = re.sub(r'http\S+' , ' ' , txt)
  txt = re.sub("[^\w]" ,' ' , txt)
  txt = wordpunct_tokenize(txt)

  Clean = [word.lower() for word in txt
           if word not in string.punctuation
           and not word.isdigit() ]

  stopwords_Removed = [word for word in Clean if word not in stopwords.words('english')]
  return stopwords_Removed

In [ ]:
data['cleaned_text'] = data['comment_text'].apply(lambda x : text_clean(x))

Stemming

In [ ]:
#import snowball stemmer package
from nltk.stem import SnowballStemmer

In [ ]:
#take an object from english stemmer
sb = SnowballStemmer('english')

In [ ]:
#define function that stamm the tokens
def stemming(txt):
    stem = [sb.stem(word) for word in txt]
    return stem
#apply function to all data
data['txtStemming']=data['cleaned_text'].apply(lambda x: stemming(x))

In [ ]:
data

In [ ]:
data['sentense'] = data['txtStemming'].apply( lambda x : ' '.join(x))

# **Bag of Words**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vect = CountVectorizer()

In [ ]:
BOW = vect.fit_transform(data['sentense'])

In [ ]:
BagOfWords = pd.DataFrame(BOW.toarray(), columns=vect.get_feature_names())

# **Word Embedding**

In [ ]:
import spacy

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
for i, row in data.iterrows():
  txt = nlp(row['sentense'])
  data.loc[i, 'Embeddings_vector'] = txt.vector